In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import Module, Conv2d, Linear, MaxPool2d, ReLU, LogSoftmax
from torch import flatten
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms.v2 as transforms
import os
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from PIL import Image
from torch.utils.data import DataLoader, random_split
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.utils import shuffle
from os import listdir
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

In [24]:
transform = transforms.Compose([transforms.ToImage(), transforms.ToDtype(torch.float32, scale=True), transforms.RandomResizedCrop((230,230))])

In [25]:
dataset = datasets.ImageFolder("C:\\Users\\aytem\\Desktop\\testgit\\5820-Final-Project\\data", transform=transform)

In [26]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

In [27]:
for images, labels in dataloader:
    print(images.shape)
    print(labels.shape)

C:\Users\aytem\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([1

In [28]:
IMG_SHAPE = (240, 240, 3) 
class BrainDetectionModel(nn.Module):
    def __init__(self, input_shape):
        super(BrainDetectionModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_shape[2], out_channels=32, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2))
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(4, 4))
        self.pool2 = nn.MaxPool2d(kernel_size=(4, 4))
        self.fc1 = nn.Linear(6272, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x

model = BrainDetectionModel(IMG_SHAPE)


In [47]:
def train_dataset(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.BCELoss()
    
    num_epochs = 250

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            print(inputs.shape)
            print(labels.shape)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(torch.squeeze(outputs.type(torch.float)), torch.squeeze(labels.type(torch.float)))  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch + 1}/10], Loss: {epoch_loss:.4f}")

        # Validation
        # model.eval()  # Set the model to evaluation mode
        # val_loss = 0.0
        # with torch.no_grad():
        #     for inputs, labels in validation_loader:
        #         outputs = model(inputs)
        #         loss = criterion(outputs, labels)
        #         val_loss += loss.item() * inputs.size(0)

        # val_loss /= len(validation_loader.dataset)
        # print(f"Validation Loss: {val_loss:.4f}")

In [48]:
train_dataset(model, dataloader)

C:\Users\aytem\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([10, 3, 230, 230])
torch.Size([10])
torch.Size([1